# Modeling

## 1: Overview: 

I completed the EDA of the financial data of Olist, a Brazilian based e-commerce company. The data was provided via kaggle and contains sales data from 2016-2018. 9 .csv files were provided with information on the products, customers and retailers on the site. After exploring the factors that drive an increase in sales (such as location, quality of product, type of product, etc), I wanted to use my knowledge of predictive modeling to predict revenue. The same SQL database from notebook "EDA" is queried here. 

## 2: Relevant Code for Preparation: 

The following code chunk(s) are provided to load the necessary packages for my analysis. 

In [43]:
# Importing Packages for work

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score, mean_absolute_error
from xgboost import XGBRegressor
from sklearn.linear_model import LogisticRegressionCV
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
import sqlite3

## 3: Linear Model: 

Below I create a linear model to serve as a baseline of accuracy. Linear models are very simple, which makes them easy to both implement and understand. However, the tradeoff of using the model is that it often lacks the accuracy and generalizability of other, more advanced models. I implement these more advanced techniques later in the analysis. 

In [36]:
#Connect to SQL database to get data for modeling
connection_to_database = sqlite3.connect("Olist_DB.db")

try:
    SQL_data = """
    SELECT *
    FROM  olist_customers_dataset
    LEFT JOIN olist_orders_dataset
    USING(customer_id)
    LEFT JOIN olist_order_items_dataset
    USING(order_id)
    LEFT JOIN olist_order_reviews_dataset
    USING(order_id)
    LEFT JOIN olist_products_dataset
    USING(product_id)
    """

    df = pd.read_sql_query(SQL_data, connection_to_database)
    
    
    
    
finally:
    connection_to_database.close()

In [37]:
# One-hot encode the 'customer_city' column
df = pd.get_dummies(df, columns=['customer_city'], prefix='city', drop_first=True)

#drop NAs and replace with median
df['review_score'] = df['review_score'].fillna(df['review_score'].median())
df['product_weight_g'] = df['product_weight_g'].fillna(df['product_weight_g'].median())
df['price'] = df['price'].fillna(df['price'].median())

In [38]:
X = df[['review_score', 'product_weight_g'] + [col for col in df.columns if col.startswith('city_')]]
y = df['price'] 

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state= 20022121)

# Initializing the Linear Regression model
linear_model = LinearRegression()

# Fitting the model
linear_model.fit(X_train, y_train)

# Making predictions
y_pred = linear_model.predict(X_test)

# Evaluating the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Displaying results
print("Mean Squared Error:", mse)
print("R-squared:", r2)

# Optional: Displaying model coefficients
coefficients = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': linear_model.coef_
})



Mean Squared Error: 1.0457834021466853e+18
R-squared: -31904766228491.55


The very large MSE score as well as a large negative R^2 indiacte this is an extremely weak model to use. It could be due to the simplicity of a linear regression, or it could be due to the variables chosen. We will refine both in the next step. 

## 4: Decision Tree: 

I now create a decision tree with slightly different variables to try to improve the model performance. Decision trees are fairly simple to understand, just like linear regression, but they tend to be more robust. I use the variables of product review, product weight, city. However, I will now determine the number of days between the purchase date of the order and the delivery date to weigh how important expediant delivery is in driving sales. 

In [39]:
df.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_state,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,...,city_xanxere,city_xapuri,city_xavantina,city_xaxim,city_xexeu,city_xinguara,city_xique-xique,city_zacarias,city_ze doca,city_zortea
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,SP,00e7ee1b050b8499577073aeb2a297a1,delivered,2017-05-16 15:05:35,2017-05-16 15:22:12,2017-05-23 10:47:57,2017-05-25 10:35:35,...,False,False,False,False,False,False,False,False,False,False
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,SP,29150127e6685892b6eab3eec79f59c7,delivered,2018-01-12 20:48:24,2018-01-12 20:58:32,2018-01-15 17:14:59,2018-01-29 12:41:19,...,False,False,False,False,False,False,False,False,False,False
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,SP,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,2018-05-19 16:07:45,2018-05-20 16:19:10,2018-06-11 14:31:00,2018-06-14 17:58:51,...,False,False,False,False,False,False,False,False,False,False
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,SP,951670f92359f4fe4a63112aa7306eba,delivered,2018-03-13 16:06:38,2018-03-13 17:29:19,2018-03-27 23:22:42,2018-03-28 16:04:25,...,False,False,False,False,False,False,False,False,False,False
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,SP,6b7d50bd145f6fc7f33cebabd7e49d0f,delivered,2018-07-29 09:51:30,2018-07-29 10:10:09,2018-07-30 15:16:00,2018-08-09 20:55:48,...,False,False,False,False,False,False,False,False,False,False


In [40]:
#convert dates to date time for calculations
df['order_approved_at'] = pd.to_datetime(df['order_approved_at'])
df['order_delivered_customer_date'] = pd.to_datetime(df['order_delivered_customer_date'])

#calculate difference between order date and delivery date
df['day_difference'] = (df['order_delivered_customer_date'] - df['order_approved_at']).dt.days

#replace missing NAs with median
df['day_difference'] = df['day_difference'].fillna(df['day_difference'].median())

In [41]:
X = df[['review_score', 'product_weight_g', 'day_difference'] + [col for col in df.columns if col.startswith('city_')]]
y = df['price'] 

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state= 20022121)

#create model
tree_model = DecisionTreeRegressor(random_state=20022121)

# Fit the model
tree_model.fit(X_train, y_train)

# Make predictions
y_pred = tree_model.predict(X_test)

# Calculate accuracy metrics
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Display results
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"R-squared (R²): {r2:.4f}")

Mean Squared Error (MSE): 44708.80
Mean Absolute Error (MAE): 79.77
R-squared (R²): -0.3640


While a negative R^2 still implies a weak model, and an absolute error of almost 80 when the mean price is 120 is pretty weak as well, it is important to note the huge improvements using a decision tree has given me. The R^2 is magnitudes lower than it was previoulsy and so is the MSE score. Futher tweaking of the model could lead to even better results.

## 4: XGBoost: 

XGBoost has become quite popular in the world of machine learning because it takes the strong results of a tree based model like above but builds the trees in parallel from "boosting", where each new tree learns from the previous one. This is also different than random forest models which build the trees independent of each other then weigh the results. Another advantage tree based modeling has over more advanced neural networks is they do not require as much data to be trained on. Our dataset has over ten thousand entries, which is a lot, but not enough to adequately train a deep learning network. The modeling is conducted below. 

In [45]:
#set variables
X = df[['review_score', 'product_weight_g', 'day_difference'] + [col for col in df.columns if col.startswith('city_')]]
y = df['price'] 

#split training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state= 20022121)

#create model
XGB_model = XGBRegressor(random_state=20022121)

# Fit the model
XGB_model.fit(X_train, y_train)

# Make predictions
y_pred = XGB_model.predict(X_test)

# Calculate accuracy metrics
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Display results
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"R-squared (R²): {r2:.4f}")

Mean Squared Error (MSE): 28496.30
Mean Absolute Error (MAE): 78.14
R-squared (R²): 0.1306


### 5: Conclusion and Next Steps

The final model, XGBoost, produced by far the best results. With the lowest MSE and MAE, it out peformed the previous two models. However, the best results have come in the form of the R^2 metric. Initially a massive negative number in the linear model, we now have a small positive value, 0.1306, as a reflection of our accuracy. Again, while this value indicates our model is far from perfect, it shows two key elements. First, that XGBoost modeling is more robust than our other two tested in this analysis. Second, it shows that the features tested (weight of product, review scores, delivery times and city) could all be huge factors in driving sales. 


For a future analysis, I would want to focus on a few more key cities, as I think the vast number of cities we had to one hot encode essentially bogs down the model and leads to overfitting. Given more time, I would also like to test a few more modeling techniques to compare them to the succes of the XGBoost. Finally, our dataset had a massive number of features, and given more time, I would like to test even more of them in the feature selection process of my analysis. 